# Validez vos dataframe avec pandera
La validatation du dataframe signifie vérifier que les données contenues dans le dataframe respectent un certains nombre de conditions. Ceci peut être utile lorsque **vous recevez régulièrement des données d'un client pour exécuter un script, ou quand vous avec développez un script sur un jeu de données et que vous devez exécuter le même script les données de la prod. Il est important que les données respectent le format défini**. 

In [1]:
# !pip install pandera

In [24]:
# packages nécessaires
import pandera as pa
import pandas as pd

In [3]:
# settings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 4000)
pd.set_option('display.max_columns', None)

In [4]:
# sample data
df = pd.DataFrame({
    'prenom': ["Hugo", "Pierre", "Anais", "Christophe", "Samira", "Sabrina"],
    'age' : [18, 25, 20, 21, 22, 24],
    'pays' : ["France", "Belgique", "Italie", "Allemagne", "France", "Italie"]
})

In [5]:
df

,prenom,age,pays
0,Hugo,18,France
1,Pierre,25,Belgique
2,Anais,20,Italie
3,Christophe,21,Allemagne
4,Samira,22,France
5,Sabrina,24,Italie


# Conditioins à vérifier

In [6]:
# définition du schéma à valider
schema = pa.DataFrameSchema({
    "age": pa.Column(int, pa.Check.less_than(26)),
    "pays": pa.Column(str, pa.Check.isin(["France", "Allemagne", "Italie", "Belgique"])),
})
# validation du schéma sur le data
schema.validate(df)

,prenom,age,pays
0,Hugo,18,France
1,Pierre,25,Belgique
2,Anais,20,Italie
3,Christophe,21,Allemagne
4,Samira,22,France
5,Sabrina,24,Italie


In [7]:
# définition du schéma à valider
schema = pa.DataFrameSchema({
    "age": pa.Column(int, pa.Check.less_than(25)),
    "pays": pa.Column(str, pa.Check.isin(["France", "Allemagne", "Italie"])),
})
# validation du schéma sur le data
try:
    schema.validate(check_obj = df, 
                    lazy=True)
except Exception as e:
    print(str(e))

A total of 2 schema errors were found.

Error Counts
------------
- schema_component_check: 2

Schema Error Summary
--------------------
                                                              failure_cases  n_failure_cases
schema_context column check                                                                 
Column         age    less_than(25)                                    [25]                1
               pays   isin({'Allemagne', 'Italie', 'France'})    [Belgique]                1

Usage Tip
---------

Directly inspect all errors by catching the exception:

```
try:
    schema.validate(dataframe, lazy=True)
except SchemaErrors as err:
    err.failure_cases  # dataframe of schema errors
    err.data  # invalid dataframe
```



In [8]:
schema

<Schema DataFrameSchema(columns={'age': <Schema Column(name=age, type=DataType(int64))>, 'pays': <Schema Column(name=pays, type=DataType(str))>}, checks=[], index=None, coerce=False, dtype=None, strict=False, name=None, ordered=False, unique_column_names=False)>

# D'autres fonctions de checks
on peut retrouver d'autres fonctions de check déjà écrites [ici](https://pandera.readthedocs.io/en/latest/reference/generated/pandera.checks.Check.html#pandera.checks.Check)

In [9]:
# définition du schéma à valider
schema = pa.DataFrameSchema({
    "age": pa.Column(int, pa.Check.le(25)),
    "pays": pa.Column(str, pa.Check.isin(["France", "Allemagne", "Italie", "Belgique"])),
})
# validation du schéma sur le data
schema.validate(df)


,prenom,age,pays
0,Hugo,18,France
1,Pierre,25,Belgique
2,Anais,20,Italie
3,Christophe,21,Allemagne
4,Samira,22,France
5,Sabrina,24,Italie


# Personaliser ses fonctions de check

In [10]:
# définition du schéma à valider
schema = pa.DataFrameSchema({
    "age": pa.Column(int, [pa.Check.le(25),
                           pa.Check(lambda age: min(age) >= 18),
                           pa.Check(lambda age: age >= 18)
                          ]),
    "pays": pa.Column(str, pa.Check.isin(["France", "Allemagne", "Italie", "Belgique"])),
})
# validation du schéma sur le data
schema.validate(df, lazy=True)


,prenom,age,pays
0,Hugo,18,France
1,Pierre,25,Belgique
2,Anais,20,Italie
3,Christophe,21,Allemagne
4,Samira,22,France
5,Sabrina,24,Italie


# Utilisation des classes pour la validation des dataframes

In [11]:
from pandera.typing import Series

class Schema(pa.SchemaModel):
    pays: Series[str] = pa.Field(isin=["France", "Allemagne", "Italie", "Belgique"])
    age: Series[int] = pa.Field(le=25)
    
    @pa.check("age")
    def price_sum_lt_20(cls, age: Series[int]) -> Series[bool]:
        return max(age) <= 25

In [12]:
Schema.validate(df)

,prenom,age,pays
0,Hugo,18,France
1,Pierre,25,Belgique
2,Anais,20,Italie
3,Christophe,21,Allemagne
4,Samira,22,France
5,Sabrina,24,Italie


# Valider les inputs de fonctions avec leurs exécutions 

## Valider l'input

In [13]:
from pandera import check_input

@check_input(schema)
def get_total_price(df: pd.DataFrame):
    return df.age.min()

In [14]:
get_total_price(df)

18

## Valider l'ouput

In [15]:
# définition du schéma à valider
output_schema = pa.DataFrameSchema({
    "age": pa.Column(int, [pa.Check.le(25),
                           pa.Check(lambda age: min(age) >= 18),
                           pa.Check(lambda age: age >= 18)
                          ]),
})

In [16]:
# dataframe à concatener avec df
dfbis = pd.DataFrame({
    'prenom': ["Carla", "Sintia", "Antoine", "Mohamed"],
    'age' : [20, 21, 22, 24],
    'pays' : ["France", "Belgique", "Italie", "Allemagne"]
})

In [17]:
from pandera import check_output

@check_output(output_schema)
def concat_df(df: pd.DataFrame, dfbis: pd.DataFrame):
    df_global = pd.concat([df, dfbis])
    return df_global

# application de la fonction
df_global = concat_df(df, dfbis)

## Valider l'input et l'output

In [18]:
from pandera import check_io
@check_io(df=schema, dfbis=schema, out=output_schema)
def combine_fruits(df: pd.DataFrame, dfbis: pd.DataFrame):
    df_global = pd.concat([df, dfbis])
    return df_global

# application
df_global = concat_df(df, dfbis)

# Doublons et valeurs manquantes
Par défaut :
- les valeurs manquantes ne sont pas acceptées (retournent une erreur)
- les doublons sont acceptés

On peut changer ces spécfications

In [19]:
# définition du schéma à valider
null_duplicate_schema = pa.DataFrameSchema({
    "age": pa.Column(int, 
                     [pa.Check.le(25),
                      pa.Check(lambda age: min(age) >= 18),
                      pa.Check(lambda age: age >= 18)],
                    nullable=True, unique=True
                    ),
})

In [20]:
# schéma acceptant les doublons, 
schema.validate(df)

,prenom,age,pays
0,Hugo,18,France
1,Pierre,25,Belgique
2,Anais,20,Italie
3,Christophe,21,Allemagne
4,Samira,22,France
5,Sabrina,24,Italie


In [21]:
# schéma refusant les doublons
try:
    null_duplicate_schema.validate(df_global)
except Exception as e:
    print(str(e))

series 'age' contains duplicate values:
0    20
1    21
2    22
3    24
Name: age, dtype: int64


In [22]:
?pa.Column

# Sauvegarder le schéma sur fichier YAML

In [23]:
from pathlib import Path

# YAML object
yaml_schema = schema.to_yaml()

# Save
file = Path("schema.yml")
file.touch()
file.write_text(yaml_schema)

C:\Users\skajie\anaconda3\lib\site-packages\pandera\schema_statistics.py:145: UserWarning: Only registered checks may be serialized to statistics. Did you forget to register it with the extension API? Check `<lambda>` will be skipped.
  warnings.warn(


478